In [ ]:
!pip install torch pandas numpy h5py tqdm

In [1]:
%load_ext autoreload
%autoreload 2

import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import h5py
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import joblib
from src.dataset import ProteinDataset
from src.model import ChemicalShiftsPredictor
from src.utils import train_model, test_model

In [27]:
# Load and prepare data
csv_file = 'data/disorder/strict.csv'
prott5_file = 'data/disorder/embeddings/unfiltered_all_prott5.h5'
prott5_res_file = 'data/disorder/embeddings/unfiltered_all_prott5_res.h5'
prostt5_file = 'data/disorder/embeddings/prostt5.h5'
chemical_shifts_df = pd.read_csv(csv_file)
chemical_shifts_df.describe()

,Unnamed: 0,entryID,stID,entity_assemID,entityID,seq_index,k,zscores,pscores,C,CA,CB,HA,H,N,HB
count,217493.000000,217493.000000,217493.000000,217493.000000,217493.000000,217493.000000,217493.000000,209016.00000,209016.000000,154402.000000,203289.000000,184908.000000,151129.000000,192492.00000,192056.000000,133093.000000
mean,950.551788,25516.888194,1.069805,1.086191,1.028737,72.199183,16.539553,11.35494,0.162650,175.888378,56.793136,38.003339,4.343245,8.27458,119.598080,2.311165
std,551.256443,10008.917334,0.445658,0.467657,0.175259,58.331021,4.661000,4.08679,0.229569,5.180103,4.960242,12.878662,0.870714,0.68850,5.324304,2.330996
min,0.000000,7349.000000,1.000000,1.000000,1.000000,1.000000,0.000000,-5.04270,-0.000000,0.000000,4.186000,-34.477000,-2.429500,-0.91400,0.000000,-1.626000
25%,470.000000,17602.000000,1.000000,1.000000,1.000000,29.000000,15.000000,10.69490,0.046000,174.517000,54.360000,30.481000,4.019000,7.88200,116.803000,1.721500
50%,943.000000,25387.000000,1.000000,1.000000,1.000000,60.000000,18.000000,12.62620,0.076600,175.996000,56.971000,34.651500,4.299000,8.27300,120.034000,2.042000
75%,1431.000000,30293.000000,1.000000,1.000000,1.000000,99.000000,21.000000,13.93570,0.136300,177.528000,59.859000,41.354000,4.619000,8.65400,122.818000,2.812500
max,1909.000000,51871.000000,8.000000,8.000000,4.000000,471.000000,21.000000,16.14970,0.999500,187.570000,176.857000,137.868000,173.538000,119.59600,181.029000,790.215500


In [28]:
#target_columns = ['C', 'CA', 'CB', 'HA', 'H', 'N', 'HB']
target_columns = ['N']
print(len(chemical_shifts_df))
chemical_shifts_df.dropna(inplace=True, subset=target_columns)
print(len(chemical_shifts_df))

# Split data into train, validation, and test sets
train_df, test_df = train_test_split(chemical_shifts_df, test_size=0.2, random_state=42)
train_df, val_df = train_test_split(train_df, test_size=0.25, random_state=42)  # 0.25 x 0.8 = 0.2

# Normalize targets based on training data statistics
scaler = StandardScaler()
train_targets = train_df[target_columns]
scaler.fit(train_targets)

# Save the mean and std for later un-normalizing
means = scaler.mean_
stds = scaler.scale_

# Save the scaler object
joblib.dump(scaler, 'scaler.joblib')

# Apply normalization to the training targets
#train_df[target_columns] = scaler.transform(train_targets)

# Apply the same normalization to validation and test sets
# val_df[target_columns] = scaler.transform(val_df[target_columns])
# test_df[target_columns] = scaler.transform(test_df[target_columns])

# Create datasets
train_dataset = ProteinDataset(target_columns, train_df, prott5_file, prott5_res_file, prostt5_file)
val_dataset = ProteinDataset(target_columns, val_df, prott5_file, prott5_res_file, prostt5_file)
test_dataset = ProteinDataset(target_columns, test_df, prott5_file, prott5_res_file, prostt5_file)

217493
192056


In [29]:
print('Trainng dataset length:', len(train_dataset))
print('Validation dataset length:', len(val_dataset))
print('Test dataset length:', len(test_dataset))

Trainng dataset length: 115233
Validation dataset length: 38411
Test dataset length: 38412


In [30]:
# Assuming you have defined model, train_loader, val_loader, test_loader
learning_rate = 0.001
weight_decay = 1e-5
patience = 10
batch_size = 1024
num_epochs = 5

trained_model = train_model(train_dataset, val_dataset, learning_rate=learning_rate, num_epochs=num_epochs, weight_decay=weight_decay, patience=patience, batch_size=batch_size, use_prostt5=False, use_protein_mean=False)
test_model(trained_model, test_dataset, batch_size=batch_size, use_prostt5=False, use_protein_mean=False)

Epoch 1/5: 100%|██████████| 113/113 [01:12<00:00,  1.55batch/s]


Epoch 1, Train Loss: 3568.5859, Validation Loss: 252.9527


Epoch 2/5: 100%|██████████| 113/113 [01:13<00:00,  1.55batch/s]


Epoch 2, Train Loss: 160.4907, Validation Loss: 108.3401


Epoch 3/5: 100%|██████████| 113/113 [01:10<00:00,  1.60batch/s]


Epoch 3, Train Loss: 77.9913, Validation Loss: 58.1687


Epoch 4/5: 100%|██████████| 113/113 [01:10<00:00,  1.60batch/s]


Epoch 4, Train Loss: 45.2795, Validation Loss: 37.5724


Epoch 5/5: 100%|██████████| 113/113 [01:11<00:00,  1.58batch/s]


Epoch 5, Train Loss: 30.6636, Validation Loss: 27.9441
Test Loss: 28.4074


In [31]:
trained_model = train_model(train_dataset, val_dataset, learning_rate=learning_rate, num_epochs=num_epochs, weight_decay=weight_decay, patience=patience, batch_size=batch_size, use_prostt5=True, use_protein_mean=False)
test_model(trained_model, test_dataset, batch_size=batch_size, use_prostt5=True, use_protein_mean=False)

Epoch 1/5:   0%|          | 0/113 [00:00<?, ?batch/s]

Epoch 1/5: 100%|██████████| 113/113 [01:09<00:00,  1.63batch/s]


Epoch 1, Train Loss: 2809.6434, Validation Loss: 133.9024


Epoch 2/5: 100%|██████████| 113/113 [01:07<00:00,  1.67batch/s]


Epoch 2, Train Loss: 75.2289, Validation Loss: 49.4624


...

Epoch 3, Train Loss: 36.5987, Validation Loss: 31.3096


Epoch 4/5: 100%|██████████| 113/113 [01:09<00:00,  1.62batch/s]


Epoch 4, Train Loss: 25.2554, Validation Loss: 24.5307


Epoch 5/5: 100%|██████████| 113/113 [01:11<00:00,  1.59batch/s]


Epoch 5, Train Loss: 19.9593, Validation Loss: 20.8266
Test Loss: 21.2452


In [32]:
trained_model = train_model(train_dataset, val_dataset, learning_rate=learning_rate, num_epochs=50, weight_decay=weight_decay, patience=patience, batch_size=batch_size, use_prostt5=True, use_protein_mean=True)
test_model(trained_model, test_dataset, batch_size=batch_size, use_prostt5=True, use_protein_mean=True)

Epoch 1/5:   0%|          | 0/113 [00:00<?, ?batch/s]

Epoch 1/5: 100%|██████████| 113/113 [01:09<00:00,  1.62batch/s]


Epoch 1, Train Loss: 2277.2400, Validation Loss: 46.7049


Epoch 2/5: 100%|██████████| 113/113 [01:13<00:00,  1.54batch/s]


Epoch 2, Train Loss: 24.2542, Validation Loss: 16.0128


...

Epoch 3, Train Loss: 13.1453, Validation Loss: 12.5087


Epoch 4/5: 100%|██████████| 113/113 [01:10<00:00,  1.59batch/s]


Epoch 4, Train Loss: 10.6051, Validation Loss: 11.2528


Epoch 5/5: 100%|██████████| 113/113 [01:09<00:00,  1.63batch/s]


Epoch 5, Train Loss: 9.2809, Validation Loss: 10.4593
Test Loss: 10.8457
